In [22]:

# Auto-reload pour développement interactif
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import json
import spacy
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import hashlib
import re
from collections import Counter
import warnings
import sys
warnings.filterwarnings('ignore')

# NLP avancé
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

# Détection de langue
from langdetect import detect, detect_langs
from langdetect.lang_detect_exception import LangDetectException

# Preprocessing texte
import unicodedata
import ftfy  # Pour corriger les encodages
from bs4 import BeautifulSoup

# Similarité et déduplication
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import faiss


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:

# Configuration
BASE_DIR = Path().resolve().parent.parent
sys.path.append(str(BASE_DIR / "src"))

# Répertoires
DATA_DIR = BASE_DIR / "data"
EXPORTS_DIR = DATA_DIR / "exports"
PROCESSED_DIR = DATA_DIR / "processed"
PROCESSED_DIR.mkdir(exist_ok=True)

# Configuration des modèles
#NLP_MODEL = "fr_core_news_lg"  # Modèle spaCy français
NLP_MODEL = "fr_core_news_sm"  # Modèle spaCy français léger pour éviter les problèmes de mémoire
EMBEDDINGS_MODEL = "paraphrase-multilingual-MiniLM-L12-v2"
SIMILARITY_THRESHOLD = 0.85  # Seuil de similarité pour déduplication

print(f"Répertoire de données: {DATA_DIR}")
print(f"Répertoire de sortie: {PROCESSED_DIR}")




Répertoire de données: C:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data
Répertoire de sortie: C:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data\processed


In [24]:
# Chargement du fichier JSON enrichi (priorité) ou brut (fallback)
enriched_file = EXPORTS_DIR / "enriched_article.json"  # MODIFIÉ: sans "s"
enriched_files_alt = EXPORTS_DIR / "enriched_articles.json"  # Alternative
raw_file = EXPORTS_DIR / "raw_articles.json"


In [25]:

# Détection automatique du fichier source - MODIFIÉE
if enriched_file.exists():
    source_file = enriched_file
    print(f" FICHIER ENRICHI DÉTECTÉ: {enriched_file}")
    print("   Mode: Preprocessing avancé sur données pré-enrichies")
elif enriched_files_alt.exists():
    source_file = enriched_files_alt
    print(f" FICHIER ENRICHI ALTERNATIF DÉTECTÉ: {enriched_files_alt}")
    print("   Mode: Preprocessing avancé sur données pré-enrichies")
elif raw_file.exists():
    source_file = raw_file
    print(f" FICHIER BRUT DÉTECTÉ: {raw_file}")
    print("   → Mode: Preprocessing complet depuis zéro")
else:
    print(f" ERREUR: Aucun fichier source trouvé!")
    print(f"   Recherche: {enriched_file} OU {raw_file}")
    print("   Solution: Exécutez d'abord collect_articles.ipynb ou enrich_articles.ipynb")
    exit(1)

with open(source_file, 'r', encoding='utf-8') as f:
    articles_data = json.load(f)





 FICHIER ENRICHI ALTERNATIF DÉTECTÉ: C:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data\exports\enriched_articles.json
   Mode: Preprocessing avancé sur données pré-enrichies


In [26]:
# Mode adaptatif selon la source - MODIFIÉ
ENRICHED_MODE = "enriched_article" in str(source_file)  # Compatible avec les deux formats

print(f" {len(articles_data)} articles chargés depuis {source_file.name}")



 200 articles chargés depuis enriched_articles.json


In [27]:
# Conversion en DataFrame pour manipulation
df = pd.DataFrame(articles_data)

In [28]:
print("\n HARMONISATION DES NOMS DE COLONNES")

# Harmonisation text_cleaned vs cleaned_text
if 'cleaned_text' in df.columns and 'text_cleaned' not in df.columns:
    df['text_cleaned'] = df['cleaned_text']
    print("    Harmonisation: cleaned_text → text_cleaned")
elif 'text' in df.columns and 'text_cleaned' not in df.columns:
    df['text_cleaned'] = df['text']
    print("    Création: text → text_cleaned")


 HARMONISATION DES NOMS DE COLONNES
    Harmonisation: cleaned_text → text_cleaned


In [29]:
# Vérification des colonnes essentielles
required_columns = ['title', 'text', 'source']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    print(f"     Colonnes manquantes: {missing_columns}")
else:
    print("    Toutes les colonnes essentielles présentes")

    Toutes les colonnes essentielles présentes


In [30]:
# Inspection rapide
print(f"\n STRUCTURE DES DONNÉES:")
print(f"   Colonnes: {list(df.columns)}")
print(f"   Articles avec texte complet: {df['text'].notna().sum()}")
print(f"   Articles sans texte: {df['text'].isna().sum()}")
if 'text_cleaned' in df.columns:
    print(f"   Articles avec texte nettoyé: {df['text_cleaned'].notna().sum()}")
print(f"   Longueur moyenne du texte: {df['text'].str.len().mean():.0f} caractères")



 STRUCTURE DES DONNÉES:
   Colonnes: ['id', 'title', 'summary', 'text', 'published', 'source', 'url', 'created_at', 'cleaned_text', 'language', 'entities', 'embedding', 'quality_score', 'text_cleaned']
   Articles avec texte complet: 200
   Articles sans texte: 0
   Articles avec texte nettoyé: 200
   Longueur moyenne du texte: 6995 caractères


In [31]:
print("\n ÉTAPE 2: Analyse adaptative des données")

# Détection des colonnes d'enrichissement déjà présentes
enrichment_columns = {
    'language': 'language' in df.columns and df['language'].notna().sum() > 0,
    'entities': 'entities' in df.columns and df['entities'].notna().sum() > 0,
    'quality_score': 'quality_score' in df.columns and df['quality_score'].notna().sum() > 0,
    'embedding': 'embedding' in df.columns and df['embedding'].notna().sum() > 0
}



 ÉTAPE 2: Analyse adaptative des données


In [32]:
print(f"\n ÉTAT DES ENRICHISSEMENTS EXISTANTS:")
for col, present in enrichment_columns.items():
    status = " Présent" if present else " Absent" 
    count = df[col].notna().sum() if present else 0
    print(f"   {col}: {status} ({count} articles)")


 ÉTAT DES ENRICHISSEMENTS EXISTANTS:
   language:  Présent (200 articles)
   entities:  Présent (200 articles)
   quality_score:  Présent (200 articles)
   embedding:  Présent (200 articles)


In [33]:
# Adaptation de la stratégie
if enrichment_columns['language'] and enrichment_columns['entities']:
    print(f"\n MODE DÉTECTÉ: Preprocessing complémentaire avancé")
    print(f"   → Focus: Déduplication, biais, corpus calibration, métriques avancées")
    SKIP_BASIC_ENRICHMENT = True
else:
    print(f"\n MODE DÉTECTÉ: Preprocessing complet depuis zéro") 
    print(f"   → Pipeline: Enrichissement + Analyses avancées")
    SKIP_BASIC_ENRICHMENT = False


 MODE DÉTECTÉ: Preprocessing complémentaire avancé
   → Focus: Déduplication, biais, corpus calibration, métriques avancées


In [34]:
print("\n ÉTAPE 3: Nettoyage avancé des données")

def clean_text_advanced(text):
    """Nettoyage robuste et avancé du texte"""
    if pd.isna(text) or not text:
        return ""
    
    # Correction encodage
    text = ftfy.fix_text(text)
    
    # Suppression HTML résiduel
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Normalisation Unicode
    text = unicodedata.normalize('NFKC', text)
    
    # Suppression caractères de contrôle
    text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    
    # Normalisation espaces
    text = re.sub(r'\s+', ' ', text)
    
    # Suppression URLs et emails
    text = re.sub(r'http[s]?://\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    
    # Suppression patterns RSS spécifiques
    text = re.sub(r'#xtor=RSS-\d+.*', '', text)
    text = re.sub(r'\[.*?\]$', '', text)  # Crédits en fin d'article
    
    return text.strip()



 ÉTAPE 3: Nettoyage avancé des données


In [35]:
# Application du nettoyage si nécessaire
if 'text_cleaned' not in df.columns or df['text_cleaned'].isna().any():
    print("    Application du nettoyage avancé...")
    df['text_cleaned'] = df['text'].apply(clean_text_advanced)
    print(f"       {len(df)} textes nettoyés")
else:
    print("    Textes déjà nettoyés détectés")



    Textes déjà nettoyés détectés


In [36]:
# Filtrage des articles trop courts ou vides
min_length = 100  # caractères minimum
df_clean = df[df['text_cleaned'].str.len() >= min_length].copy()
print(f"    Filtrage longueur minimum ({min_length} chars): {len(df)} → {len(df_clean)} articles")



    Filtrage longueur minimum (100 chars): 200 → 200 articles


In [37]:
print("\nÉTAPE 4: Gestion intelligente de la langue")

if not enrichment_columns['language'] or not SKIP_BASIC_ENRICHMENT:
    print("    Détection de langue en cours...")
    
    def detect_language_robust(text):
        """Détection de langue avec fallback"""
        if not text or len(text) < 50:
            return 'unknown', 0.0
        
        try:
            # langdetect avec probabilités
            langs = detect_langs(text)
            primary_lang = langs[0]
            return primary_lang.lang, primary_lang.prob
        except LangDetectException:
            # Fallback: détection basique
            try:
                return detect(text), 0.5
            except:
                return 'unknown', 0.0

    # Application de la détection
    language_results = df_clean['text_cleaned'].apply(detect_language_robust)
    df_clean['language'] = [result[0] for result in language_results]
    df_clean['language_confidence'] = [result[1] for result in language_results]
    
    print(f"    Détection de langue terminée")
else:
    print("     Langues déjà détectées, validation des données...")
    if 'language_confidence' not in df_clean.columns:
        df_clean['language_confidence'] = df_clean['language'].apply(lambda x: 0.9 if x == 'fr' else 0.7)


ÉTAPE 4: Gestion intelligente de la langue
     Langues déjà détectées, validation des données...


In [38]:
# Analyse des langues détectées
lang_counts = df_clean['language'].value_counts()
print(f"\n DISTRIBUTION DES LANGUES:")
for lang, count in lang_counts.head(5).items():
    pct = count / len(df_clean) * 100
    print(f"   {lang}: {count} articles ({pct:.1f}%)")


 DISTRIBUTION DES LANGUES:
   fr: 130 articles (65.0%)
   en: 70 articles (35.0%)


In [39]:
# Sélection intelligente selon la distribution
if lang_counts.get('fr', 0) > len(df_clean) * 0.3:  # Si >30% en français
    df_filtered = df_clean[df_clean['language'] == 'fr'].copy()
    print(f"   🇫🇷 Focus français: {len(df_filtered)} articles sélectionnés")
else:
    # Garder top 2 langues si pas assez de français
    top_langs = lang_counts.head(2).index.tolist()
    df_filtered = df_clean[df_clean['language'].isin(top_langs)].copy()
    print(f"    Multi-langues: {len(df_filtered)} articles ({top_langs})")



   🇫🇷 Focus français: 130 articles sélectionnés


In [40]:

# Chargement du modèle d'embeddings
print("    Chargement du modèle sentence-transformers...")
embeddings_model = SentenceTransformer(EMBEDDINGS_MODEL)

def deduplicate_semantic(df, threshold=0.85):
    """Déduplication sémantique avancée avec FAISS"""
    
    print(f"    Génération des embeddings pour {len(df)} articles...")
    
    # Utilisation des embeddings existants ou génération
    if 'embedding' in df.columns and df['embedding'].notna().sum() > 0:
        print("       Utilisation des embeddings existants")
        embeddings = []
        for idx, emb in df['embedding'].items():
            if isinstance(emb, (list, np.ndarray)) and len(emb) > 0:
                embeddings.append(np.array(emb))
            else:
                # Génération pour les embeddings manquants
                text = df.loc[idx, 'text_cleaned']
                embeddings.append(embeddings_model.encode(text))
        embeddings = np.array(embeddings)
    else:
        print("       Génération des embeddings...")
        texts = df['text_cleaned'].tolist()
        embeddings = embeddings_model.encode(texts, show_progress_bar=True)
    
    # Configuration FAISS
    print("     Configuration de l'index FAISS...")
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)  # Inner Product (cosine après normalisation)
    
    # Normalisation pour cosine similarity
    faiss.normalize_L2(embeddings)
    index.add(embeddings.astype('float32'))
    
    # Recherche des doublons
    print("    Recherche des doublons sémantiques...")
    similarities, indices = index.search(embeddings.astype('float32'), k=5)  # Top 5 similaires
    
    to_remove = set()
    duplicate_pairs = []
    
    for i, (sim_scores, sim_indices) in enumerate(zip(similarities, indices)):
        for j, (score, idx) in enumerate(zip(sim_scores, sim_indices)):
            if j > 0 and score > threshold and idx not in to_remove and i not in to_remove:
                # Garde le plus récent ou le mieux noté
                if df.iloc[i].get('quality_score', 0) >= df.iloc[idx].get('quality_score', 0):
                    to_remove.add(idx)
                else:
                    to_remove.add(i)
                
                duplicate_pairs.append((i, idx, score))
    
    # Suppression des doublons
    df_dedup = df.drop(df.index[list(to_remove)]).copy()
    
    print(f"    Résultats déduplication:")
    print(f"      Articles originaux: {len(df)}")
    print(f"      Doublons détectés: {len(to_remove)}")
    print(f"      Articles finaux: {len(df_dedup)}")
    print(f"      Taux de déduplication: {len(to_remove)/len(df)*100:.1f}%")
    
    # Exemples de doublons détectés
    if duplicate_pairs:
        print(f"    Exemples de doublons détectés:")
        for i, (idx1, idx2, sim) in enumerate(duplicate_pairs[:3]):
            title1 = df.iloc[idx1]['title'][:50]
            title2 = df.iloc[idx2]['title'][:50]
            print(f"      {i+1}. Similarité {sim:.3f}:")
            print(f"         A: {title1}...")
            print(f"         B: {title2}...")
    
    return df_dedup, embeddings

# Application de la déduplication
df_clean_dedup, article_embeddings = deduplicate_semantic(df_filtered, SIMILARITY_THRESHOLD)




    Chargement du modèle sentence-transformers...


OSError: Le fichier de pagination est insuffisant pour terminer cette opération. (os error 1455)

In [ ]:
print("\n ÉTAPE 6: Gestion avancée des entités nommées")

if not enrichment_columns['entities'] or not SKIP_BASIC_ENRICHMENT:
    print("    Extraction complète des entités avec spaCy...")
    
    # Chargement du modèle spaCy français
    print(f"       Chargement du modèle spaCy: {NLP_MODEL}")
    try:
        nlp = spacy.load(NLP_MODEL)
    except OSError:
        print(f"       Modèle {NLP_MODEL} non trouvé. Installation...")
        import subprocess
        subprocess.run(f"python -m spacy download {NLP_MODEL}", shell=True)
        nlp = spacy.load(NLP_MODEL)

    def extract_entities_advanced(text, nlp_model):
        """Extraction d'entités avec enrichissements"""
        if not text or len(text) < 50:
            return {
                'persons': [], 'organizations': [], 'locations': [],
                'dates': [], 'money': [], 'misc': []
            }
        
        # Traitement avec spaCy (limiter la longueur pour performance)
        doc = nlp_model(text[:8000])  # Premier 8k caractères
        
        entities = {
            'persons': [],
            'organizations': [],
            'locations': [],
            'dates': [],
            'money': [],
            'misc': []
        }
        
        for ent in doc.ents:
            entity_text = ent.text.strip()
            if len(entity_text) < 2:  
                continue
                
            if ent.label_ in ['PERSON']:
                entities['persons'].append(entity_text)
            elif ent.label_ in ['ORG']:
                entities['organizations'].append(entity_text)
            elif ent.label_ in ['GPE', 'LOC']:
                entities['locations'].append(entity_text)
            elif ent.label_ in ['DATE', 'TIME']:
                entities['dates'].append(entity_text)
            elif ent.label_ in ['MONEY']:
                entities['money'].append(entity_text)
            else:
                entities['misc'].append(entity_text)
        
        # Déduplication et nettoyage
        for key in entities:
            entities[key] = list(set(entities[key]))  # Suppression doublons
            entities[key] = [e for e in entities[key] if len(e) > 1]  # Filtrage longueur
        
        return entities

    # Application de l'extraction d'entités
    entities_results = []
    for text in tqdm(df_clean_dedup['text_cleaned'], desc="Extraction NER"):
        entities = extract_entities_advanced(text, nlp)
        entities_results.append(entities)

    # Ajout des résultats au DataFrame
    df_clean_dedup['entities_advanced'] = entities_results
    
    print(f"       Extraction NER terminée")
    
else:
    print("    Amélioration des entités existantes...")
    
    def improve_entities(existing_entities):
        """Amélioration et nettoyage des entités existantes"""
        if not existing_entities or not isinstance(existing_entities, dict):
            return {
                'persons': [], 'organizations': [], 'locations': [],
                'dates': [], 'money': [], 'misc': []
            }
        
        improved = {
            'persons': [],
            'organizations': [],
            'locations': [],
            'dates': [],
            'money': [],
            'misc': []
        }
        
        # Nettoyage et déduplication
        for key in improved.keys():
            if key in existing_entities and isinstance(existing_entities[key], list):
                # Nettoyage des entités
                cleaned = [str(e).strip() for e in existing_entities[key] if e and len(str(e)) > 1]
                # Déduplication case-insensitive
                seen = set()
                for entity in cleaned:
                    if entity.lower() not in seen:
                        improved[key].append(entity)
                        seen.add(entity.lower())
        
        return improved
    
    # Application de l'amélioration
    df_clean_dedup['entities_advanced'] = df_clean_dedup['entities'].apply(improve_entities)




In [ ]:
# Création de colonnes métriques enrichies
df_clean_dedup['persons_count'] = df_clean_dedup['entities_advanced'].apply(lambda x: len(x.get('persons', [])))
df_clean_dedup['organizations_count'] = df_clean_dedup['entities_advanced'].apply(lambda x: len(x.get('organizations', [])))
df_clean_dedup['locations_count'] = df_clean_dedup['entities_advanced'].apply(lambda x: len(x.get('locations', [])))
df_clean_dedup['entities_total'] = (df_clean_dedup['persons_count'] + 
                                   df_clean_dedup['organizations_count'] + 
                                   df_clean_dedup['locations_count'])


In [ ]:
print(f"\n    STATISTIQUES ENTITÉS AVANCÉES:")
print(f"      Moyenne personnes/article: {df_clean_dedup['persons_count'].mean():.1f}")
print(f"      Moyenne organisations/article: {df_clean_dedup['organizations_count'].mean():.1f}")
print(f"      Moyenne lieux/article: {df_clean_dedup['locations_count'].mean():.1f}")
print(f"      Moyenne total entités/article: {df_clean_dedup['entities_total'].mean():.1f}")



In [ ]:
# Top entités par catégorie
print(f"\n    TOP ENTITÉS DÉTECTÉES:")
all_persons = [p for entities in df_clean_dedup['entities_advanced'] for p in entities.get('persons', [])]
all_orgs = [o for entities in df_clean_dedup['entities_advanced'] for o in entities.get('organizations', [])]
all_locs = [l for entities in df_clean_dedup['entities_advanced'] for l in entities.get('locations', [])]

if all_persons:
    top_persons = Counter(all_persons).most_common(3)
    print(f"      Personnes: {', '.join([f'{p} ({c})' for p, c in top_persons])}")

if all_orgs:
    top_orgs = Counter(all_orgs).most_common(3)
    print(f"      Organisations: {', '.join([f'{o} ({c})' for o, c in top_orgs])}")

if all_locs:
    top_locs = Counter(all_locs).most_common(3)
    print(f"      Lieux: {', '.join([f'{l} ({c})' for l, c in top_locs])}")




In [ ]:
print("\n ÉTAPE 7: Segmentation sémantique avancée")

# Téléchargement des ressources NLTK si nécessaire
try:
    import ssl
    _create_unverified_https_context = ssl._create_unverified_context
    ssl._create_default_https_context = _create_unverified_https_context
except:
    pass

nltk.download('punkt', quiet=True)

def segment_text_advanced(text):
    """Segmentation en phrases avec analyse sémantique avancée"""
    if not text or len(text) < 100:
        return {
            'sentences': [],
            'sentence_count': 0,
            'avg_sentence_length': 0,
            'paragraphs': [],
            'paragraph_count': 0,
            'text_complexity': 0,
            'readability_score': 0
        }
    
    # Segmentation en phrases (multi-langue)
    sentences = sent_tokenize(text[:5000], language='french')  # Limiter pour performance
    
    # Segmentation en paragraphes
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip() and len(p) > 20]
    
    # Métriques avancées
    sentence_lengths = [len(s.split()) for s in sentences]
    avg_sentence_length = np.mean(sentence_lengths) if sentence_lengths else 0
    
    # Score de complexité basé sur la longueur des phrases
    complexity = 0
    if sentence_lengths:
        variance = np.var(sentence_lengths)
        long_sentences = sum(1 for length in sentence_lengths if length > 20)
        complexity = min((variance / 100) + (long_sentences / len(sentences)), 1.0)
    
    # Score de lisibilité approximatif (Flesch-like)
    if sentence_lengths and avg_sentence_length > 0:
        readability = max(0, min(1, 1 - (avg_sentence_length - 10) / 20))
    else:
        readability = 0.5
    
    return {
        'sentences': sentences[:50],  # Limiter pour stockage
        'sentence_count': len(sentences),
        'avg_sentence_length': avg_sentence_length,
        'paragraphs': paragraphs[:20],  # Limiter pour stockage
        'paragraph_count': len(paragraphs),
        'text_complexity': complexity,
        'readability_score': readability
    }

In [ ]:
# Application de la segmentation
print("    Segmentation avancée en cours...")
segmentation_results = []

for text in tqdm(df_clean_dedup['text_cleaned'], desc="Segmentation"):
    segments = segment_text_advanced(text)
    segmentation_results.append(segments)

df_clean_dedup['segmentation'] = segmentation_results

In [ ]:
# Extraction des métriques de segmentation
df_clean_dedup['sentence_count'] = df_clean_dedup['segmentation'].apply(lambda x: x['sentence_count'])
df_clean_dedup['paragraph_count'] = df_clean_dedup['segmentation'].apply(lambda x: x['paragraph_count'])
df_clean_dedup['avg_sentence_length'] = df_clean_dedup['segmentation'].apply(lambda x: x['avg_sentence_length'])
df_clean_dedup['text_complexity'] = df_clean_dedup['segmentation'].apply(lambda x: x['text_complexity'])
df_clean_dedup['readability_score'] = df_clean_dedup['segmentation'].apply(lambda x: x['readability_score'])



In [ ]:
print(f"\n    STATISTIQUES SEGMENTATION AVANCÉES:")
print(f"      Moyenne phrases/article: {df_clean_dedup['sentence_count'].mean():.1f}")
print(f"      Moyenne paragraphes/article: {df_clean_dedup['paragraph_count'].mean():.1f}")
print(f"      Longueur moyenne phrases: {df_clean_dedup['avg_sentence_length'].mean():.1f} mots")
print(f"      Score complexité moyen: {df_clean_dedup['text_complexity'].mean():.2f}")
print(f"      Score lisibilité moyen: {df_clean_dedup['readability_score'].mean():.2f}")


In [ ]:
print("\n ÉTAPE 8: Analyse des biais temporels et géographiques")

# Conversion et nettoyage des dates
print("    Analyse temporelle...")
df_clean_dedup['published_clean'] = pd.to_datetime(df_clean_dedup['published'], errors='coerce')

# Extraction des composants temporels
df_clean_dedup['hour'] = df_clean_dedup['published_clean'].dt.hour
df_clean_dedup['day_of_week'] = df_clean_dedup['published_clean'].dt.day_name()
df_clean_dedup['month'] = df_clean_dedup['published_clean'].dt.month
df_clean_dedup['date_only'] = df_clean_dedup['published_clean'].dt.date


In [ ]:
# Analyse des biais temporels
valid_dates = df_clean_dedup[df_clean_dedup['published_clean'].notna()]

if len(valid_dates) > 0:
    print(f"       Analyse sur {len(valid_dates)} articles avec dates valides")
    
    # Distribution horaire
    hour_dist = valid_dates['hour'].value_counts().head(3)
    print(f"       Heures de publication principales:")
    for hour, count in hour_dist.items():
        pct = count / len(valid_dates) * 100
        print(f"         {hour}h: {count} articles ({pct:.1f}%)")
    
    # Distribution par jour
    day_dist = valid_dates['day_of_week'].value_counts().head(3)
    print(f"       Jours de publication principaux:")
    for day, count in day_dist.items():
        pct = count / len(valid_dates) * 100
        print(f"         {day}: {count} articles ({pct:.1f}%)")
    
    # Calcul du score de biais temporel
    hour_entropy = -sum((p := hour_dist / len(valid_dates)) * np.log2(p + 1e-10))
    day_entropy = -sum((p := day_dist / len(valid_dates)) * np.log2(p + 1e-10))
    
    # Normalisation (entropie max = log2(24) pour heures, log2(7) pour jours)
    hour_bias = 1 - (hour_entropy / np.log2(24))  # 0 = uniforme, 1 = très biaisé
    day_bias = 1 - (day_entropy / np.log2(7))
    
    print(f"       Scores de biais temporel:")
    print(f"         Biais horaire: {hour_bias:.2f} (0=uniforme, 1=concentré)")
    print(f"         Biais quotidien: {day_bias:.2f}")


In [ ]:
# Analyse géographique via entités lieux
print("\n    Analyse géographique...")
all_locations = []
location_counts_by_article = []

for entities in df_clean_dedup['entities_advanced']:
    article_locations = entities.get('locations', [])
    location_counts_by_article.append(len(article_locations))
    all_locations.extend(article_locations)

location_distribution = Counter(all_locations)
df_clean_dedup['locations_count'] = location_counts_by_article

if location_distribution:
    print(f"       Lieux les plus mentionnés:")
    for location, count in location_distribution.most_common(5):
        pct = count / len(all_locations) * 100 if len(all_locations) > 0 else 0
        print(f"         {location}: {count} mentions ({pct:.1f}%)")
    
    # Score de biais géographique
    if len(location_distribution) > 1:
        geo_probs = np.array(list(location_distribution.values())) / len(all_locations)
        geo_entropy = -sum(geo_probs * np.log2(geo_probs + 1e-10))
        max_entropy = np.log2(min(len(location_distribution), 50))  # Entropie max théorique
        geo_bias = 1 - (geo_entropy / max_entropy) if max_entropy > 0 else 0
        
        print(f"       Score de biais géographique: {geo_bias:.2f}")
        print(f"         (0=distribution équilibrée, 1=concentration forte)")
    else:
        geo_bias = 1.0  # Maximum bias if only one location
        print(f"        Biais géographique maximal détecté")

In [ ]:
# Ajout des scores de biais au DataFrame
df_clean_dedup['temporal_bias_hour'] = hour_bias if 'hour_bias' in locals() else 0
df_clean_dedup['temporal_bias_day'] = day_bias if 'day_bias' in locals() else 0
df_clean_dedup['geographic_bias'] = geo_bias if 'geo_bias' in locals() else 0


In [ ]:
def calculate_advanced_quality_score(row):
    """Calcul d'un score de qualité multi-dimensionnel"""
    scores = {}
    
    # 1. Score de longueur (0-1)
    text_len = len(str(row.get('text_cleaned', '')))
    scores['length'] = min(text_len / 2000, 1.0)  # Optimal à 2000 caractères
    
    # 2. Score d'entités (0-1)
    entities_count = row.get('entities_total', 0)
    scores['entities'] = min(entities_count / 10, 1.0)  # Optimal à 10 entités
    
    # 3. Score de lisibilité (0-1)
    scores['readability'] = row.get('readability_score', 0.5)
    
    # 4. Score de complexité inversé (0-1)
    complexity = row.get('text_complexity', 0.5)
    scores['complexity'] = 1 - complexity  # Moins complexe = meilleur
    
    # 5. Score de structure (0-1)
    sentence_count = row.get('sentence_count', 0)
    paragraph_count = row.get('paragraph_count', 0)
    if sentence_count > 0 and paragraph_count > 0:
        structure_ratio = min(sentence_count / paragraph_count, 10) / 10  # Ratio phrases/paragraphes
        scores['structure'] = structure_ratio
    else:
        scores['structure'] = 0.1
    
    # 6. Score de langue (0-1)
    lang_confidence = row.get('language_confidence', 0.5)
    scores['language'] = lang_confidence
    
    # Score global pondéré
    weights = {
        'length': 0.2,
        'entities': 0.25, 
        'readability': 0.2,
        'complexity': 0.15,
        'structure': 0.1,
        'language': 0.1
    }
    
    final_score = sum(scores[key] * weights[key] for key in scores)
    
    return {
        'quality_score_advanced': final_score,
        'quality_breakdown': scores
    }


In [ ]:
# Application du calcul de qualité
print("    Calcul des scores de qualité...")
quality_results = []

for _, row in tqdm(df_clean_dedup.iterrows(), total=len(df_clean_dedup), desc="Qualité"):
    quality_result = calculate_advanced_quality_score(row)
    quality_results.append(quality_result)



In [ ]:
# Ajout des résultats
df_clean_dedup['quality_score_advanced'] = [r['quality_score_advanced'] for r in quality_results]
df_clean_dedup['quality_breakdown'] = [r['quality_breakdown'] for r in quality_results]

print(f"\n    STATISTIQUES QUALITÉ AVANCÉES:")
print(f"      Score moyen: {df_clean_dedup['quality_score_advanced'].mean():.3f}")
print(f"      Score médian: {df_clean_dedup['quality_score_advanced'].median():.3f}")
print(f"      Score min: {df_clean_dedup['quality_score_advanced'].min():.3f}")
print(f"      Score max: {df_clean_dedup['quality_score_advanced'].max():.3f}")


In [ ]:
# Distribution par quartiles
quartiles = df_clean_dedup['quality_score_advanced'].quantile([0.25, 0.5, 0.75])
print(f"      Quartiles: Q1={quartiles[0.25]:.3f}, Q2={quartiles[0.5]:.3f}, Q3={quartiles[0.75]:.3f}")



In [ ]:


def create_stratified_calibration_corpus(df, n_samples=300):
    """Création d'un corpus stratifié pour calibration"""
    
    # Définition des strates multi-dimensionnelles
    print("   Définition des strates...")
    
    # 1. Strate par qualité (3 niveaux)
    quality_tertiles = df['quality_score_advanced'].quantile([0.33, 0.67])
    df['quality_stratum'] = pd.cut(df['quality_score_advanced'], 
                                  bins=[0, quality_tertiles[0.33], quality_tertiles[0.67], 1],
                                  labels=['low', 'medium', 'high'])
    
    # 2. Strate par longueur (3 niveaux)
    df['text_length'] = df['text_cleaned'].str.len()
    length_tertiles = df['text_length'].quantile([0.33, 0.67])
    df['length_stratum'] = pd.cut(df['text_length'],
                                 bins=[0, length_tertiles[0.33], length_tertiles[0.67], float('inf')],
                                 labels=['short', 'medium', 'long'])
    
    # 3. Strate par richesse en entités (3 niveaux)
    if df['entities_total'].max() > 0:
        entity_tertiles = df['entities_total'].quantile([0.33, 0.67])
        df['entity_stratum'] = pd.cut(df['entities_total'],
                                     bins=[-1, entity_tertiles[0.33], entity_tertiles[0.67], float('inf')],
                                     labels=['sparse', 'moderate', 'rich'])
    else:
        df['entity_stratum'] = 'sparse'
    
    # 4. Strate par source (top sources + autres)
    source_counts = df['source'].value_counts()
    top_sources = source_counts.head(5).index.tolist()
    df['source_stratum'] = df['source'].apply(lambda x: x if x in top_sources else 'other')
    
    print(f"      Strates créées:")
    print(f"         Qualité: {df['quality_stratum'].value_counts().to_dict()}")
    print(f"         Longueur: {df['length_stratum'].value_counts().to_dict()}")
    print(f"         Entités: {df['entity_stratum'].value_counts().to_dict()}")
    print(f"         Sources: {len(df['source_stratum'].unique())} catégories")
    
    # Échantillonnage stratifié proportionnel
    print("   Échantillonnage stratifié...")
    
    # Groupement par strates multiples
    strata_cols = ['quality_stratum', 'length_stratum', 'entity_stratum', 'source_stratum']
    grouped = df.groupby(strata_cols, group_keys=False)
    
    # Calcul des tailles d'échantillon par strate
    strata_sizes = grouped.size()
    total_size = len(df)
    
    sample_dfs = []
    remaining_samples = n_samples
    
    for stratum, group in grouped:
        if remaining_samples <= 0:
            break
            
        # Taille proportionnelle de l'échantillon pour cette strate
        stratum_size = len(group)
        proportion = stratum_size / total_size
        target_sample_size = max(1, int(proportion * n_samples))
        
        # Ajustement si on dépasse le nombre d'échantillons restants
        actual_sample_size = min(target_sample_size, remaining_samples, stratum_size)
        
        if actual_sample_size > 0:
            # Échantillonnage au sein de la strate
            if len(group) >= actual_sample_size:
                # Tri par score de qualité pour prendre les meilleurs
                group_sorted = group.sort_values('quality_score_advanced', ascending=False)
                stratum_sample = group_sorted.head(actual_sample_size)
                sample_dfs.append(stratum_sample)
                remaining_samples -= actual_sample_size
    
    # Combinaison des échantillons de toutes les strates
    if sample_dfs:
        calibration_corpus = pd.concat(sample_dfs, ignore_index=True)
    else:
        # Fallback: échantillonnage simple par qualité
        calibration_corpus = df.nlargest(n_samples, 'quality_score_advanced')
    
    # Complément aléatoire si nécessaire
    if len(calibration_corpus) < n_samples:
        remaining_df = df[~df.index.isin(calibration_corpus.index)]
        if len(remaining_df) > 0:
            additional_samples = min(n_samples - len(calibration_corpus), len(remaining_df))
            additional = remaining_df.sample(n=additional_samples, random_state=42)
            calibration_corpus = pd.concat([calibration_corpus, additional], ignore_index=True)
    
    return calibration_corpus.head(n_samples)  



In [ ]:
# Création du corpus de calibration
calibration_corpus = create_stratified_calibration_corpus(df_clean_dedup, n_samples=300)

print(f"\n    CORPUS DE CALIBRATION CRÉÉ:")
print(f"      Taille finale: {len(calibration_corpus)} articles")
print(f"      Score qualité moyen: {calibration_corpus['quality_score_advanced'].mean():.3f}")




In [ ]:
# Analyse de la représentativité
print(f"\n    REPRÉSENTATIVITÉ DU CORPUS:")
print(f"      Sources: {calibration_corpus['source'].nunique()} uniques")
print(f"      Langues: {calibration_corpus['language'].value_counts().to_dict()}")




In [ ]:
# Top sources dans le corpus
source_dist = calibration_corpus['source'].value_counts().head(5)
print(f"      Top sources:")
for source, count in source_dist.items():
    pct = count / len(calibration_corpus) * 100
    source_name = source.split('/')[-1] if '/' in source else source
    print(f"         {source_name}: {count} articles ({pct:.1f}%)")

# Distribution qualité
quality_dist = calibration_corpus['quality_stratum'].value_counts()
print(f"      Distribution qualité: {quality_dist.to_dict()}")


In [ ]:
# Métriques de qualité finales
quality_metrics = {
    'source_file': str(source_file.name),
    'enriched_mode': ENRICHED_MODE,
    'total_articles_input': len(articles_data),
    'articles_after_deduplication': len(df_clean_dedup),
    'calibration_corpus_size': len(calibration_corpus),
    'deduplication_rate': ((len(df) - len(df_clean_dedup)) / len(df)) if len(df) > 0 else 0,
    'avg_quality_score': df_clean_dedup['quality_score_advanced'].mean(),
    'language_distribution': df_clean_dedup['language'].value_counts().to_dict(),
    'entities_avg_per_article': df_clean_dedup['entities_total'].mean(),
    'temporal_bias_detected': df_clean_dedup['temporal_bias_hour'].iloc[0] if len(df_clean_dedup) > 0 else 0,
    'geographic_bias_detected': df_clean_dedup['geographic_bias'].iloc[0] if len(df_clean_dedup) > 0 else 0,
    'processing_timestamp': datetime.now().isoformat()
}

In [ ]:
# Sauvegarde du DataFrame principal (format optimisé)
output_file = PROCESSED_DIR / "articles_preprocessed_advanced.pkl"
df_clean_dedup.to_pickle(output_file)
print(f"    Dataset principal sauvegardé: {output_file}")


In [ ]:
# Sauvegarde du corpus de calibration
calibration_file = PROCESSED_DIR / "calibration_corpus_stratified.pkl"
calibration_corpus.to_pickle(calibration_file)
print(f"    Corpus de calibration sauvegardé: {calibration_file}")


In [ ]:
# Sauvegarde des métriques
metrics_file = PROCESSED_DIR / "advanced_preprocessing_metrics.json"
with open(metrics_file, 'w', encoding='utf-8') as f:
    json.dump(quality_metrics, f, indent=2, ensure_ascii=False, default=str)
print(f"    Métriques sauvegardées: {metrics_file}")





In [ ]:
# Export CSV léger pour analyse externe
csv_file = PROCESSED_DIR / "articles_preprocessed_summary.csv"
df_export = df_clean_dedup[[
    'title', 'source', 'published', 'language', 'quality_score_advanced',
    'entities_total', 'sentence_count', 'readability_score', 'text_complexity'
]].copy()
df_export.to_csv(csv_file, index=False, encoding='utf-8')
print(f"    Export CSV résumé: {csv_file}")





In [ ]:
# Export JSON du corpus de calibration 
calibration_json = PROCESSED_DIR / "calibration_corpus_300.json"
calibration_export = calibration_corpus[[
    'id', 'title', 'text_cleaned', 'source', 'published', 'language',
    'quality_score_advanced', 'entities_advanced'
]].to_dict('records')

with open(calibration_json, 'w', encoding='utf-8') as f:
    json.dump(calibration_export, f, ensure_ascii=False, indent=2, default=str)
print(f"    Corpus calibration JSON: {calibration_json}")


In [ ]:
# Résumé des performances
processing_time = (datetime.now() - datetime.fromisoformat(quality_metrics['processing_timestamp'])).total_seconds()

print(f" RÉSULTATS FINAUX:")
print(f"   Source: {quality_metrics['source_file']}")
print(f"   Mode: {'Enrichissement complémentaire' if ENRICHED_MODE else 'Pipeline complet'}")
print(f"   Articles traités: {quality_metrics['total_articles_input']}")
print(f"   Articles finaux: {quality_metrics['articles_after_deduplication']}")
print(f"   Corpus de calibration: {quality_metrics['calibration_corpus_size']}")
print(f"   Taux de déduplication: {quality_metrics['deduplication_rate']:.1%}")

In [ ]:
print(f"\n MÉTRIQUES DE QUALITÉ:")
print(f"   Score qualité moyen: {quality_metrics['avg_quality_score']:.3f}")
print(f"   Entités par article: {quality_metrics['entities_avg_per_article']:.1f}")
print(f"   Biais temporel détecté: {quality_metrics['temporal_bias_detected']:.2f}")
print(f"   Biais géographique: {quality_metrics['geographic_bias_detected']:.2f}")

In [ ]:
print(f"\n DISTRIBUTION LINGUISTIQUE:")
for lang, count in quality_metrics['language_distribution'].items():
    pct = count / quality_metrics['articles_after_deduplication'] * 100
    print(f"   {lang}: {count} articles ({pct:.1f}%)")


In [ ]:
print(f"\nFICHIERS GÉNÉRÉS:")
print(f"   1. {output_file.name} - Dataset principal avec preprocessing avancé")
print(f"   2. {calibration_file.name} - Corpus stratifié pour calibration")
print(f"   3. {metrics_file.name} - Métriques détaillées")
print(f"   4. {csv_file.name} - Export CSV pour analyse")


In [ ]:
print(f"\nINSTRUCTIONS D'INSTALLATION SI ERREURS:")
print(f"   Si erreur spaCy: !python -m spacy download fr_core_news_lg")
print(f"   Si erreur FAISS: !pip install faiss-cpu")
print(f"   Si erreur sentence-transformers: !pip install sentence-transformers")
print(f"   Si erreur langdetect: !pip install langdetect")
print(f"   Si erreur ftfy: !pip install ftfy")